# Twitter Analysis
## Group 7

In [1]:
import graphlab as gl
import numpy as np
import pandas as pd
import json
import re

import pyLDAvis 
import pyLDAvis.graphlab

In [4]:
file_path = './cleaned_tweets.csv'
df = pd.read_csv(file_path)
df = df.rename(columns={'Unnamed: 0':'user_id','0':'tweets'})
sf = gl.SFrame(df)

This non-commercial license of GraphLab Create for academic use is assigned to Y.Li-25@student.tudelft.nl and will expire on February 22, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\xps\AppData\Local\Temp\graphlab_server_1492022893.log.0


In [28]:
# read in previous twe data
file_path = './cleaned_london_timelines_twitter_data_3000t10000us.json'
#file_path = './london_users_data_3200us1000_en_txt.json'
json_file = open(file_path)
tweets = json.load(json_file)
for k in tweets.keys():
    if tweets[k]=={}:
        del tweets[k]
    else:
        t = tweets[k]
        tweets[k] = {}
        tweets[k]['tweets'] = [t['text'] for t in t['tweets']]
tweets_df = pd.DataFrame.from_dict(tweets,orient='index')

In [29]:
#tweets[u'315405726']['tweets'][0]
tweets_df['tweets'][0]

[u'#tenerife #lasamericas #friends @ Las Americas Beach https://t.co/dNNflpMPmW',
 u'#tenerife @ Las Americas Beach https://t.co/WlqD9Ozvbb',
 u"RT @markagrantham: For anyone that is into their dance music and DJ's would seriously consider follwing @SwitchDiscoUK on here and FB. Top\u2026",
 u'Back at the mighty SWX Bristol tonight for Farfetched Friday 24th March in the main room!\n\n#switchdisco https://t.co/pYSA23CJ6p',
 u'Saturday night business, tunes till 4am!\n\n#switchdisco \u2014 listening to Switch Disco at Sakura Reading https://t.co/km1bBzWXNC',
 u'He smashed it #switchdisco #clubnights #england #uk #stpatricksday #swxbristol @ SWX Bristol https://t.co/8phRjRBDSH',
 u'The first ever event to take place at one of the most beautiful beaches in the whole world!Zante Flair Open 2017... https://t.co/UpQOAZROwL',
 u'#Brighton #England @ Hove Beach https://t.co/7dm0j2usCY',
 u'#Brighton #uk #england @ Brighton https://t.co/CodmvjXWiY',
 u'Gonna be a big one tonight \U0001f64c\n\n#

## important! 
Before cleaning, get some statistics!
* total tweets proccessed
* average length

In [42]:
# todo
tweets_df['user_id'] = tweets_df.index
tweets_sf = gl.SFrame(tweets_df)

In [43]:
def get_feature(row):
    feature=''
    if row['tweets']!=None:
        for t in row['tweets']:
            feature += t
    return feature
tweets_sf['tweets'] = tweets_sf.apply(get_feature)
sf = tweets_sf

In [45]:
# get valid English wordss
sf['tweets'] = sf['tweets'].apply(lambda x: re.sub("[^a-zA-Z]", " ", x))

In [46]:
# tokenization
docs = gl.text_analytics.tokenize(sf['tweets'])

In [47]:
# lemmatization
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
docs_lemmatized = docs.apply(lambda x: [wordnet_lemmatizer.lemmatize(w) for w in x])

In [54]:
# Bag-of-words
docs_cleaned = gl.text_analytics.count_words(docs_lemmatized)
# Remove stop words
docs_cleaned = docs_cleaned.dict_trim_by_keys(gl.text_analytics.stopwords(), exclude=True)
# remove some words
twitter_words ={'http','wa','don','ve'}
docs_cleaned = docs_cleaned.dict_trim_by_keys(twitter_words, exclude=True)
# Remove less freq words
docs_cleaned = docs_cleaned.dict_trim_by_values(2)

In [57]:
# create a topic model 
topic_model = gl.topic_model.create(docs_cleaned,num_topics=30, num_iterations=300)

Learning a topic model

Number of documents      1007

Vocabulary size     38206

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

...

| 270       | 8.08s         | 5287232        | 0               |

| 280       | 8.37s         | 4.44048e+006   | 0               |

| 290       | 8.64s         | 1.05731e+007   | 0               |

| 300       | 8.92s         | 5287232        | 0               |

+-----------+---------------+----------------+-----------------+

In [58]:
for i in range(30):
    print 'topic ',i,topic_model.get_topics(num_words=10,output_type='topic_words')[i]['words']

topic  0 ['colliers', 'cloud', 'market', 'hadoop', 'xjyxue', 'energy', 'property', 'climate', 'iot', 'price']
topic  1 ['share', 'sign', 'petition', 'leeds', 'giraffe', 'pal', 'conservation', 'church', 'kensaye', 'green']
topic  2 ['art', 'beauty', 'photo', 'week', 'day', 'makeup', 'shoot', 'model', 'review', 'pr']
topic  3 ['rt', 'amp', 'day', 'life', 'love', 'good', 'happy', 'girl', 'people', 'man']
topic  4 ['se', 'interior', 'railway', 'site', 'virgin', 'visit', 'bright', 'trainsec', 'ln', 'gin']
topic  5 ['day', 'time', 'people', 'ha', 'love', 'thing', 'make', 'amp', 'work', 'back']
topic  6 ['data', 'latest', 'design', 'social', 'business', 'company', 'tech', 'google', 'marketing', 'brand']
topic  7 ['rt', 'amp', 'great', 'uk', 'today', 'team', 'year', 'work', 'event', 'people']
topic  8 ['london', 'greater', 'amp', 'house', 'station', 'time', 'park', 'city', 'shoreditch', 'singapore']
topic  9 ['st', 'dear', 'graham', 'youhadonej', 'pixoto', 'shepherdsmkts', 'scala', 'startisans

In [51]:
# todo: 
topic_model['topics'][0]

{'topic_probabilities': array('d', [9.366804046461801e-06, 1.2144765606026082e-05, 1.3150973172017127e-05, 4.958866204830583e-07, 1.3080444735123633e-05, 9.23275782476463e-06, 1.2807377049182969e-05, 5.912262031452815e-06, 1.0098969905072188e-05, 1.1534025374858671e-05, 1.2102142079150525e-05, 1.1591515011014476e-05, 1.0461345329011533e-05, 0.00018663348738006893, 1.3360053440216061e-05, 1.2993762993765857e-05, 1.255808112520646e-05, 1.2487512487515423e-05, 1.1124707976418254e-05, 9.493971328208958e-06, 1.196458482890933e-05, 1.1742602160641608e-05, 1.2738853503187418e-05, 6.938180808991463e-06, 1.8653584286221844e-06, 1.3395847287343322e-05, 2.597942429594871e-06, 1.2980269989618567e-05, 5.4546446299017405e-06, 1.2764871074804533e-05, 1.312163758037267e-05, 1.2368583797157598e-05, 9.406895254226135e-07, 2.639149138316891e-06, 1.281886937572349e-05, 9.630200308168902e-06, 2.7646457106511775e-06, 8.58663918942358e-06, 6.170554115759813e-06, 1.2162490878134264e-05, 1.338329764454215e-05,

In [59]:
pyLDAvis.enable_notebook()
pyLDAvis.graphlab.prepare(topic_model, docs_cleaned)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
17     17.310982        1       1 -0.379584 -0.062791
5      12.911156        1       2 -0.300240 -0.073972
29      8.006517        1       3 -0.272327  0.097145
7       5.966230        1       4 -0.249866  0.105026
16      5.809238        1       5 -0.199249  0.082587
6       3.740749        1       6 -0.008614  0.064493
24      3.294310        1       7  0.003191 -0.070299
11      3.215019        1       8 -0.027572 -0.273208
18      3.099062        1       9  0.028404  0.063250
8       3.059679        1      10 -0.023666 -0.214150
3       3.040484        1      11 -0.171434  0.068761
19      2.534192        1      12  0.056234 -0.059100
2       2.154803        1      13  0.051006 -0.002834
14      2.019564        1      14  0.072236  0.048588
21      1.812505        1      15  0.082332 -0.012377
25      1.767330        1      16  0.081857  0.030507
12      1.661905        1      17  0.061444 -0.022576
22      1.577367        1      18  0.087489  0.034147
13      1.503631        1      19  0.092394  0.009194
26      1.458064        1      20  0.098036  0.015471
0       1.448374        1      21  0.096322  0.014095
27      1.445937        1      22  0.100159  0.033596
15      1.443600        1      23  0.094009  0.015269
20      1.439326        1      24  0.085753  0.022330
1       1.412827        1      25  0.093860  0.032255
4       1.398438        1      26  0.094507 -0.002108
23      1.384270        1      27  0.095596  0.012899
9       1.374931        1      28  0.096311  0.010752
28      1.357082        1      29  0.082399  0.003543
10      1.352428        1      30  0.079011  0.029510, topic_info=      Category          Freq             Term         Total  loglift  logprob
term                                                                         
19081  Default  21102.000000               rt  21102.000000  30.0000  30.0000
26292  Default   5190.000000           london   5190.000000  29.0000  29.0000
3334   Default   6890.000000              amp   6890.000000  28.0000  28.0000
35392  Default   5666.000000              day   5666.000000  27.0000  27.0000
9752   Default   3099.000000           people   3099.000000  26.0000  26.0000
9935   Default   3802.000000             love   3802.000000  25.0000  25.0000
1783   Default   1257.000000            video   1257.000000  24.0000  24.0000
2782   Default   2579.000000            night   2579.000000  23.0000  23.0000
30395  Default   3318.000000            great   3318.000000  22.0000  22.0000
34763  Default   3765.000000             good   3765.000000  21.0000  21.0000
7598   Default   4498.000000            today   4498.000000  20.0000  20.0000
10646  Default   2412.000000             week   2412.000000  19.0000  19.0000
14056  Default   3797.000000               ha   3797.000000  18.0000  18.0000
20708  Default   2176.000000               uk   2176.000000  17.0000  17.0000
35454  Default   4608.000000             time   4608.000000  16.0000  16.0000
9808   Default   1766.000000            happy   1766.000000  15.0000  15.0000
1989   Default   1857.000000          tonight   1857.000000  14.0000  14.0000
15816  Default   2739.000000             back   2739.000000  13.0000  13.0000
23032  Default   1046.000000             game   1046.000000  12.0000  12.0000
36501  Default   1058.000000            trump   1058.000000  11.0000  11.0000
9651   Default   1706.000000             life   1706.000000  10.0000  10.0000
12364  Default    958.000000             live    958.000000   9.0000   9.0000
25749  Default    712.000000          greater    712.000000   8.0000   8.0000
16852  Default    860.000000            photo    860.000000   7.0000   7.0000
33976  Default   2546.000000             work   2546.000000   6.0000   6.0000
76     Default    815.000000            music    815.000000   5.0000   5.0000
24695  Default   1290.000000             show